In [3]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

## Initial Experiments

In [ ]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [ ]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [ ]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

In [ ]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [ ]:
print(subset_loc_clone)

## Final Code

In [48]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [49]:
len(p1)

270297

In [ ]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [ ]:
regex

In [ ]:
len(p1)

In [ ]:
locations = []
for r in tqdm(p1['regex_sent']):
    try:
        locations.append([match.span() \
                  for match in re.finditer(re.compile("|".join(regex["REGEX"]), 
                                                  re.IGNORECASE),r)])
    except:
        locations.append([])
        
p1["pruned_regex_location"] = locations

In [ ]:
len(p1)

In [ ]:
p1 = p1[p1["pruned_regex_location"].str.len() != 0]
len(p1)

In [ ]:
total_merged = []

for index, row in p1.iterrows():
    curr = str(row["regex_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    #print("CURR: ", len(curr))
    # print(curr)
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(curr) == 2 and isinstance(curr[1], int)):
        start = curr[0]
        end = curr[1]
        #print(start, end)
        length = len(row["NoteTXT"])
        if (start < 100):
            start = 100
        elif (end > length - 100):
            end = length - 100
        curr = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(curr)):
            start = curr[j][0]
            end = curr[j][1]
            loc_list.append((start, end))
        length = len(row["NoteTXT"])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        curr = loc_list
        
    for interval in curr:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
            
    total_merged.append(merged)
p1["merged_row_location"] = total_merged

In [ ]:
p1

In [ ]:
total_merged_note_txt = []
for index, row in p1.iterrows():
    curr = row["merged_row_location"]
    merged_note_txt = ""
    
    for i in range(len(curr)):
        start = curr[i][0]
        end = curr[i][1]
        
        merged_note_txt += " ------- " + row["NoteTXT"][start:end]
    total_merged_note_txt.append(merged_note_txt)
    
p1["merged_regex_sent"] = total_merged_note_txt

In [ ]:
len(p1)

In [ ]:
p1['merged_regex_sent'] = p1['merged_regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [ ]:
#p1['NoteTXT'] = p1['NoteTXT'].astype(str)
p1 = p1.drop_duplicates(subset='merged_regex_sent', keep="first")
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)

In [ ]:
p1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated_pruned.csv", index = False)

## Merging Regex matches 

In [106]:
seq = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_updated_pruned.csv")

In [107]:
l = []
for note in tqdm(seq["merged_regex_sent"]):
    try:
        m = list(set(re.findall(re.compile("|".join(regex["REGEX"]), re.IGNORECASE), note)))
        m = list(set(map(str.lower, m)))
        l.append(m)
    except:
        l.append([])

seq["merged_regex_match"] = l

100%|█████████████████████████████████████████████████████████████████████████| 60871/60871 [00:03<00:00, 15928.03it/s]


In [108]:
seq[seq["merged_regex_match"].str.len() == 0]

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,...,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent,merged_regex_match


In [109]:
seq.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_complete.csv")

## Getting #  of Unique Encounters per Patient

In [51]:
## getting number of unique encounters per patient
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated_pruned.csv")

In [52]:
len(df)

62617

In [32]:
df = df.drop_duplicates(subset = 'PatientEncounterID', keep = "first")

In [33]:
summary_stats_1 = df.groupby(['EMPI'])["PatientEncounterID"].count().reset_index()
summary_stats_1

,PatientID,PatientEncounterID
0,Z12913885,8
1,Z17028069,9
2,Z6352398,10
3,Z6352614,6
4,Z6352700,5
...,...,...
2773,Z8712370,9
2774,Z8713474,1
2775,Z8713489,1
2776,Z8713893,18


In [23]:
summary_stats_1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_summary_stats_updated.csv")

In [24]:
s1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_summary_stats_updated.csv")
s2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\apoe_2_summary_stats_updated.csv")
s3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\apoe_3_summary_stats_updated.csv")
s4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_summary_stats_updated.csv")
s5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_summary_stats_updated.csv")

In [25]:
len(s1)

2778

In [26]:
len(s2)

3335

In [27]:
len(s3)

3499

In [28]:
len(s4)

3431

In [29]:
len(s5)

3918

In [36]:
2778+3335+3499+3431+3918

16961

In [34]:
frames = [s1, s2, s3, s4, s5]
total = pd.concat(frames)

In [35]:
total.reset_index(drop = True)

,Unnamed: 0,EMPI,PatientEncounterID
0,0,100000272,10
1,1,100000521,6
2,2,100000623,5
3,3,100000894,7
4,4,100000920,3
...,...,...,...
16956,3913,113714475,7
16957,3914,113716850,4
16958,3915,113722466,5
16959,3916,113722742,12


In [38]:
total.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated_7_5_2021.csv")

In [79]:
subset = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated_pruned.csv")

In [80]:
subset = subset.iloc[0:5000]
subset

,Unnamed: 0,Unnamed: 0.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,...,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent
0,0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,...,4.532520e+08,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1,"[(100, 109)]","[(5750, 5958)]",------- ncer. She passed secondary to this. He...
1,1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,...,4.924833e+08,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1,"[(100, 107)]","[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...
2,14,14,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,2370049295,Progress Notes,...,2.338373e+09,1.0,----------------------------------------------...,"[(8741, 8749)]","'ractures, obesity, insulin resistance, dyslip...",['dementia'],1,"[(100, 109)]","[(8641, 8849)]","------- ractures, obesity, insulin resistance,..."
3,15,15,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,4959788710,Progress Notes,...,4.951007e+09,1.0,----------------------------------------------...,"[(1048, 1052), (4878, 4887), (5104, 5110), (51...",' accept for this week when he ran out and use...,['MOCA'],1,"[(100, 105)]","[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...
4,20,20,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,Telemedicine - audio only,4570521668,Progress Notes,...,4.558686e+09,1.0,----------------------------------------------...,"[(5467, 5476), (5693, 5699), (5703, 5712)]",'ue Tylenol 1g q8 as needed - If pain worsens ...,['cognition'],1,"[(100, 110)]","[(5367, 5576), (5593, 5812)]",------- ue Tylenol 1g q8 as needed - If pain w...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,20191,20191,100025967,324799,Z6385322,3.214874e+09,2018-08-27 00:00:00.0000000,Hospital Encounter,2113081341,Consults,...,2.074567e+09,1.0,----------------------------------------------...,"[(549, 557), (5416, 5424)]","'ine syndrome. However, he underwent 3 separat...",['cerebral'],1,"[(100, 109)]","[(449, 657), (5316, 5524)]","------- ine syndrome. However, he underwent 3 ..."
4996,20193,20193,100025967,324799,Z6385322,3.214874e+09,2018-08-27 00:00:00.0000000,Hospital Encounter,2113151794,Consults,...,2.074639e+09,1.0,----------------------------------------------...,"[(2379, 2388), (2604, 2610)]",'story Denies Patient Goals: reduce pain Preca...,['Cognition'],1,"[(100, 110)]","[(2279, 2488), (2504, 2710)]",------- story Denies Patient Goals: reduce pai...
4997,20195,20195,100025967,324799,Z6385322,3.214874e+09,2018-08-27 00:00:00.0000000,Hospital Encounter,2114136885,H&P,...,2.075792e+09,1.0,----------------------------------------------...,"[(4450, 4458)]",'sonally reviewed the social history. Family H...,['Dementia'],1,"[(100, 109)]","[(4350, 4558)]",------- sonally reviewed the social history. F...
4998,20196,20196,100025967,324799,Z6385322,3.214874e+09,2018-08-27 00:00:00.0000000,Hospital Encounter,2114747673,Progress Notes,...,2.076280e+09,1.0,----------------------------------------------...,"[(3893, 3901)]",' Calcium Headaches • Baclofen Rash • Gabapent...,['Dementia'],1,"[(100, 109)]","[(3793, 4001)]",------- Calcium Headaches • Baclofen Rash • G...


In [81]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [82]:
regex.head()

,REGEX
0,\sdementia
1,\scognition
2,\smemory
3,\sCDR
4,\sMMSE


In [83]:
l = []
for note in tqdm(subset["merged_regex_sent"]):
    try:
        m = list(set(re.findall(re.compile("|".join(regex["REGEX"]), re.IGNORECASE), note)))
        m = list(set(map(str.lower, m)))
        l.append(m)
    except:
        l.append([])

subset["merged_regex_match"] = l

100%|███████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 18400.43it/s]


In [84]:
a = ["cognition", "Cognition"]
a = list(set(map(str.lower, a)))
a

['cognition']

In [85]:
subset.head()

,Unnamed: 0,Unnamed: 0.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,...,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent,merged_regex_match
0,0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,...,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1,"[(100, 109)]","[(5750, 5958)]",------- ncer. She passed secondary to this. He...,[ cerebral]
1,1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,...,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1,"[(100, 107)]","[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...,[ memory]
2,14,14,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,2370049295,Progress Notes,...,1.0,----------------------------------------------...,"[(8741, 8749)]","'ractures, obesity, insulin resistance, dyslip...",['dementia'],1,"[(100, 109)]","[(8641, 8849)]","------- ractures, obesity, insulin resistance,...",[ dementia]
3,15,15,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,4959788710,Progress Notes,...,1.0,----------------------------------------------...,"[(1048, 1052), (4878, 4887), (5104, 5110), (51...",' accept for this week when he ran out and use...,['MOCA'],1,"[(100, 105)]","[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...,"[ moca, cognition, memory]"
4,20,20,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,Telemedicine - audio only,4570521668,Progress Notes,...,1.0,----------------------------------------------...,"[(5467, 5476), (5693, 5699), (5703, 5712)]",'ue Tylenol 1g q8 as needed - If pain worsens ...,['cognition'],1,"[(100, 110)]","[(5367, 5576), (5593, 5812)]",------- ue Tylenol 1g q8 as needed - If pain w...,"[ cognition, memory]"


In [87]:
subset.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated_v2_snippet.csv")

## Cleaning up Similar Encounters on Patient Level

In [86]:
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated_pruned.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\MIND_DS\\Dropbox (Partners HealthCare)\\NLP\\Tanish\\BigDataSets\\Regex_match\\4\\apoe_4_matches_updated_pruned_2.csv'

In [2]:
df.head()

NameError: name 'df' is not defined

In [ ]:
# total_merged_sequences = []

# for i in tqdm(unique_empi):
#     merged_sequences = df[df["EMPI"] == i]["merged_regex_sent"].values
#     #print("MERGED: ", merged_sequences, "\n")

#     counter = 0
#     for j in merged_sequences:
# #         choices = []
        
# #         for k in merged_sequences:
# #             if (k != j):
# #                 choices.append(k)
# #         choices = merged_sequences[0:counter]
# #         choices += merged_sequences[counter+1:]
#         #print("\n------------------")
#         choices = list(merged_sequences[0:counter]) + list(merged_sequences[counter+1:])
#         result = get_match(j, choices)
#         score = result[0][1]
#         txt = result[0][0]
        
#         if (score >= 90):
#             # 90 is duplicate string threshold
#             idx = df[df["merged_regex_sent"] == str(txt)].index[0]
#             df.drop([idx])
        
#         counter += 1

In [ ]:
len(df)

In [ ]:
df["PatientEncounterID"].nunique()

In [ ]:
df["PatientEncounterID"].isna().sum()

In [ ]:
df = df.drop_duplicates(subset='merged_regex_sent', keep="first")

In [ ]:
len(df)

## Data Visualization

In [46]:
updated = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated.csv")

In [47]:
len(updated)

78368

In [ ]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

In [ ]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

In [ ]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])

In [ ]:
'at sleeping with his legs elevated on a pillow at night does help his leg swelling. Recommended Triamcinolone 0.1% ointment every morning and Mupirocin ointment at night, risks/benefits reviewed. Path'